In [1]:
import pygame
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# 2. Constantes do Jogo (Cores, Tamanhos, Velocidades, etc.)
# Cores (RGB)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0) # Cor do Pac-Man
# ... (outras cores para fantasmas, etc.)

# Configurações do Labirinto
CELL_SIZE = 40 # Tamanho de cada célula (bloco) em pixels
# ... (outras constantes de tamanho se necessário)

# Configurações do Pac-Man
PACMAN_RADIUS = CELL_SIZE // 2
PACMAN_SPEED = 2 # Velocidade em pixels por frame
# ... (outras constantes do Pac-Man)

# Configurações dos Fantasmas (opcional, pode ser adicionado depois)
GHOST_SPEED = 1
# ...



In [3]:
# 3. Definição do Labirinto (Matriz 2D)
# 1 = Parede, 0 = Caminho, 2 = Pílula, 3 = Pílula de Poder (exemplo)
level = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 2, 2, 2, 0, 0, 0, 2, 2, 1],
    [1, 2, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 0, 0, 0, 0, 1, 2, 1],
    [1, 2, 1, 0, 1, 1, 0, 1, 2, 1],
    [1, 2, 1, 0, 0, 0, 0, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
]

# Calcular as dimensões da tela com base no labirinto
SCREEN_WIDTH = len(level[0]) * CELL_SIZE
SCREEN_HEIGHT = len(level) * CELL_SIZE


In [4]:
# 4. Classes do Jogo (PacMan, Ghost, etc.)
# Esta é a sua classe PacMan que já definimos:
class PacMan:
    def __init__(self, start_x_grid, start_y_grid, radius, speed, cell_size):
        # ... (código do __init__ do PacMan)
        self.grid_x = start_x_grid
        self.grid_y = start_y_grid
        self.pixel_x = self.grid_x * cell_size + cell_size // 2
        self.pixel_y = self.grid_y * cell_size + cell_size // 2
        self.radius = radius
        self.color = YELLOW
        self.speed = speed
        self.cell_size = cell_size
        self.dx = 0
        self.dy = 0
        self.desired_dx = 0
        self.desired_dy = 0

    def set_direction(self, dx, dy):
        # ... (código do set_direction do PacMan)
        self.desired_dx = dx * self.speed
        self.desired_dy = dy * self.speed

    def update(self, game_level):
        # ... (código do update do PacMan, incluindo a lógica de colisão e virada)
        next_pixel_x = self.pixel_x + self.dx
        next_pixel_y = self.pixel_y + self.dy

        next_grid_x = int(next_pixel_x // self.cell_size)
        next_grid_y = int(next_pixel_y // self.cell_size)
        
        can_move_x = True
        if self.dx != 0:
            test_x = next_grid_x
            test_y = int(self.pixel_y // self.cell_size)
            if not (0 <= test_y < len(game_level) and 0 <= test_x < len(game_level[0]) and game_level[test_y][test_x] != 1):
                can_move_x = False

        can_move_y = True
        if self.dy != 0:
            test_x = int(self.pixel_x // self.cell_size)
            test_y = next_grid_y
            if not (0 <= test_y < len(game_level) and 0 <= test_x < len(game_level[0]) and game_level[test_y][test_x] != 1):
                can_move_y = False
        
        if self.desired_dx != 0:
            if abs(self.pixel_y - (self.grid_y * self.cell_size + self.cell_size // 2)) < self.speed:
                test_grid_y_for_desired = int(self.pixel_y // self.cell_size)
                test_grid_x_for_desired = int((self.pixel_x + self.desired_dx) // self.cell_size)
                
                if (0 <= test_grid_y_for_desired < len(game_level) and
                    0 <= test_grid_x_for_desired < len(game_level[0]) and
                    game_level[test_grid_y_for_desired][test_grid_x_for_desired] != 1):
                    self.dx = self.desired_dx
                    self.dy = 0
                    self.pixel_y = self.grid_y * self.cell_size + self.cell_size // 2

        if self.desired_dy != 0:
            if abs(self.pixel_x - (self.grid_x * self.cell_size + self.cell_size // 2)) < self.speed:
                test_grid_x_for_desired = int(self.pixel_x // self.cell_size)
                test_grid_y_for_desired = int((self.pixel_y + self.desired_dy) // self.cell_size)

                if (0 <= test_grid_y_for_desired < len(game_level) and
                    0 <= test_grid_x_for_desired < len(game_level[0]) and
                    game_level[test_grid_y_for_desired][test_grid_x_for_desired] != 1):
                    self.dy = self.desired_dy
                    self.dx = 0
                    self.pixel_x = self.grid_x * self.cell_size + self.cell_size // 2

        if can_move_x:
            self.pixel_x = next_pixel_x
        else:
            self.dx = 0

        if can_move_y:
            self.pixel_y = next_pixel_y
        else:
            self.dy = 0
            
        self.grid_x = int(self.pixel_x // self.cell_size)
        self.grid_y = int(self.pixel_y // self.cell_size)


    def draw(self, screen):
        # ... (código do draw do PacMan)
        pygame.draw.circle(screen, self.color, (int(self.pixel_x), int(self.pixel_y)), self.radius)

# (Futuramente, adicione classes para Fantasmas, Pílulas, Power-Ups, etc., aqui)

In [5]:
# 5. Inicialização do Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Pac-Man em Python")
clock = pygame.time.Clock() # Para controlar o FPS

In [6]:
# 6. Criação de Instâncias dos Objetos do Jogo
# Crie seu Pac-Man
pacman = PacMan(1, 1, PACMAN_RADIUS, PACMAN_SPEED, CELL_SIZE)
# (Futuramente, crie instâncias de fantasmas, etc.)

In [ ]:
# 7. Loop Principal do Jogo
running = True
while running:
    # 7.1. Processamento de Eventos (Entrada do Usuário, Fechar Janela)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False # Encerra o loop principal
        if event.type == pygame.KEYDOWN:
            # Controle do Pac-Man com as setas
            if event.key == pygame.K_LEFT:
                pacman.set_direction(-1, 0)
            elif event.key == pygame.K_RIGHT:
                pacman.set_direction(1, 0)
            elif event.key == pygame.K_UP:
                pacman.set_direction(0, -1)
            elif event.key == pygame.K_DOWN:
                pacman.set_direction(0, 1)

    # 7.2. Atualização do Estado do Jogo (Lógica do Jogo)
    pacman.update(level)
    # (Futuramente, atualize fantasmas, verifique colisões Pac-Man/pílula, Pac-Man/fantasma, etc.)

    # 7.3. Desenho na Tela
    screen.fill(BLACK) # Limpa a tela a cada frame (fundo preto)

    # Desenhar o labirinto
    for row_index, row in enumerate(level):
        for col_index, cell in enumerate(row):
            x = col_index * CELL_SIZE
            y = row_index * CELL_SIZE
            if cell == 1: # Parede
                pygame.draw.rect(screen, BLUE, (x, y, CELL_SIZE, CELL_SIZE))
            elif cell == 2: # Pílula
                pygame.draw.circle(screen, WHITE, (x + CELL_SIZE // 2, y + CELL_SIZE // 2), CELL_SIZE // 4)
            # (Futuramente, adicione desenho para pílulas de poder, etc.)

    # Desenhar o Pac-Man
    pacman.draw(screen)
    # (Futuramente, desenhe fantasmas, pontuação, vidas, etc.)

    # 7.4. Atualizar a Exibição (Mostrar tudo que foi desenhado)
    pygame.display.flip()

    # 7.5. Controle de FPS (Quadros por Segundo)
    clock.tick(60) # Limita o jogo a 60 FPS


In [ ]:
# 8. Finalização do Pygame (Após o Loop Principal)
pygame.quit()
sys.exit() # Encerra o programa Python completamente